## Netconf scrapli plays

Lets start with helper functions:  
- *ypath2xml* - converts yangpaths to xml rpc
- *ppxml* - pretty prints xml


In [2]:
import re
import xml.dom.minidom

def ypath2xml(ypath, xmlns='', operation=None):
    #transforms xpath-like string (ypath) e.g. "/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni=10444"
    #to xml-like sequence of elements tags: 
    # <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    #   <eps-items>
    #       <epId-items>
    #           <Ep-list>
    #               <epId>1</epId>
    #               <nws-items>
    #                   <vni-items>
    #                       <Nw-list operation="remove">
    #                           <vni>10444</vni>
    #                       </Nw-list>
    #                   </vni-items>
    #               </nws-items>
    #           </Ep-list>
    #       </epId-items>
    #   </eps-items>
    # </System>
    #
    # it has optional parameter 'operation' that adds the string 'operation="value"' to the element marked with square brackets '[]'
    # for an nxos the value of the operation can be either "remove" or "replace"for an nxo
    #
    # there is a problem with parsing elements that contain the '/' separator within like this tDn element has:
    # /System/intf-items/svi-items/If-list/rtvrfMbr-items/tDn=/System/inst-items/Inst-list[name='{vrf_name}']
    # the same case if you want to configure a physical interface like Eth103/1/20
    # 
    # as a workaround I mark all the slashes with additional one, then replace that doubleslashes with # sign
    # then split ypath and unmark them back

    ypath = re.sub(r'//', '#', ypath) # <-- replace doubleslashes with '#'
    pl = ypath.split('/')
    
    xmls = f'<{pl[1]} xmlns="{xmlns}">' if xmlns else f'<{pl[1]}>'
    xmle = f'</{pl[1]}>'
     
    def _ypath2xml(pl):
        key = ''
        xmls = ''
        xmle = ''
        operation_set = ("merge", "replace", "create", "delete", "remove")        

        for i in range(len(pl)):
            elem = pl[i]
            if "=" in elem:
                elem,key = elem.split("=", 1)
                key = re.sub(r'#', '/', key) # <-- replace '#'
                xmls += f'<{elem}>{key}</{elem}>'
                break
            if "[]" in elem:
                elem = elem[:-2]
                if operation:
                    if operation not in operation_set:
                        raise ValueError(f'Incorrect operation value\nmust be one of the following: {", ".join(operation_set)}')
                    xmls += f'<{elem} operation="{operation}">'                
                else:
                    xmls += f'<{elem}>'
            else:                    
                xmls += f'<{elem}>'
            xmle = f'</{elem}>' + xmle
     
        if key and i < len(pl)-1:
            return xmls + _ypath2xml(pl[(i+1)::]) + xmle #recursion
        else:
            return xmls + xmle
         
    return xmls + _ypath2xml(pl[2::]) + xmle



def ppxml(xmlstr):
    print(xml.dom.minidom.parseString(xmlstr).toprettyxml(indent="    "))

### Adding a Decorator  
the decorator augments the function by adding outer tags and making multiple ypath processing

In [1]:
from functools import wraps

def wrap_in_tag(tag, xmlns=None):
    xmls = f'<{tag} xmlns="{xmlns}">' if xmlns else f'<{tag}>'
    xmle = f'</{tag}>'    
    
    def decorator(func):        
        @wraps(func)
        def wrapped(*args, **kwargs):       
            if isinstance(args[0], str): 
                return xmls + f'{func(*args, **kwargs)}' + xmle
            if isinstance(args[0], list):
                return xmls + ''.join([func(ypath, **kwargs) for ypath in args[0]]) + xmle
            else:
                raise ValueError('ypath arg should be string or list of strings')
        return wrapped
    return decorator

@wrap_in_tag("config")
def ypath_config(ypath, xmlns='', operation=None):
    return ypath2xml(ypath, xmlns, operation)

@wrap_in_tag("System", "http://cisco.com/ns/yang/cisco-nx-os-device")
def ypath_system(ypath, xmlns='', operation=None):
    return ypath2xml(ypath, xmlns, operation)

### Creating VLAN with SVI and IPv4 Address
Note: the switch in question is the public Cisco Nexus Sandbox

In [12]:
from scrapli_netconf.driver import NetconfScrape
my_device = {
    "host": "sbx-nxos-mgmt.cisco.com",
    "port": 10000,
    "auth_username": "admin",
    "auth_password": "Admin_1234!",
    "auth_strict_key": False,    
}

vlan_id = 1234
vlan_name = "VL1234"
vrf_name = "Tenant-1"
mtu = 9000
description = "configured by netconf"
ip_address = "10.0.12.34//24"

vlan_get_path = f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}"
vlan_config_path = vlan_get_path + f"/name={vlan_name}/id={vlan_id}/BdState=active/adminSt=active/bridgeMode=mac/fwdCtrl=mdst-flood/fwdMode=bridge,route/mode=CE"

svi_get_path = f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
svi_config_path = svi_get_path + f"/mtu={mtu}/descr={description}/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='{vrf_name}']"

ipv4_get_path = f"/ipv4-items/inst-items/dom-items/Dom-list/name={vrf_name}/if-items/If-list[]/id=vlan{vlan_id}"
ipv4_config_path = ipv4_get_path + f"/addr-items/Addr-list/addr={ip_address}"

ypath_config = [vlan_config_path, svi_config_path, ipv4_config_path]

config_xml = "<config>" + ypath_system(ypath_config, operation="replace") + "</config>"
ppxml(config_xml)

with NetconfScrape(**my_device) as conn:
    response = conn.edit_config(config=config_xml, target="running")
    print(response.result)


<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bd-items>
            <bd-items>
                <BD-list operation="replace">
                    <fabEncap>vlan-1234</fabEncap>
                    <name>VL1234</name>
                    <id>1234</id>
                    <BdState>active</BdState>
                    <adminSt>active</adminSt>
                    <bridgeMode>mac</bridgeMode>
                    <fwdCtrl>mdst-flood</fwdCtrl>
                    <fwdMode>bridge,route</fwdMode>
                    <mode>CE</mode>
                </BD-list>
            </bd-items>
        </bd-items>
        <intf-items>
            <svi-items>
                <If-list operation="replace">
                    <id>vlan1234</id>
                    <mtu>9000</mtu>
                    <descr>configured by netconf</descr>
                    <adminSt>up</adminSt>
                    <rtvrfMbr-items>
                        <tDn>/System/in

### tidy up
clear all the configuration we have made

In [13]:
ypath_get = [vlan_get_path, svi_get_path, ipv4_get_path]

delete_xml = "<config>" + ypath_system(ypath_get, operation="remove") + "</config>"
ppxml(delete_xml)

with NetconfScrape(**my_device) as conn:
    response = conn.edit_config(config=delete_xml, target="running")
    print(response.result)


<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bd-items>
            <bd-items>
                <BD-list operation="remove">
                    <fabEncap>vlan-1234</fabEncap>
                </BD-list>
            </bd-items>
        </bd-items>
        <intf-items>
            <svi-items>
                <If-list operation="remove">
                    <id>vlan1234</id>
                </If-list>
            </svi-items>
        </intf-items>
        <ipv4-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>Tenant-1</name>
                        <if-items>
                            <If-list operation="remove">
                                <id>vlan1234</id>
                            </If-list>
                        </if-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </ipv4-items>
    </System>
<